In [3]:
import numpy as np
import time

In [4]:
N = 6
a = np.arange(N).reshape((2,3))
b = np.arange(5)
c = np.arange(7)

print(a)
print(b)
print(c)

print(a.shape)
print(b.shape)
print(c.shape)

[[0 1 2]
 [3 4 5]]
[0 1 2 3 4]
[0 1 2 3 4 5 6]
(2, 3)
(5,)
(7,)


In [41]:
d1 = b[:, None] * c[None, :]
d2 = b[None, :] * c[:, None]

print(d1.shape)
print(d2.shape)

print("d1", d1)
print()
print("d2", d2)

(5, 7)
(7, 5)
d1 [[ 0  0  0  0  0  0  0]
 [ 0  1  2  3  4  5  6]
 [ 0  2  4  6  8 10 12]
 [ 0  3  6  9 12 15 18]
 [ 0  4  8 12 16 20 24]]

d2 [[ 0  0  0  0  0]
 [ 0  1  2  3  4]
 [ 0  2  4  6  8]
 [ 0  3  6  9 12]
 [ 0  4  8 12 16]
 [ 0  5 10 15 20]
 [ 0  6 12 18 24]]


In [44]:
e1 = a[:, :, None] * c[None, None, :]
e2 = a[:, None, :] * c[None, :, None]
e3 = a[None, :, :] * c[:, None, None]

print(a.shape)
print(c.shape)

print("a")
print(a)
print("c")
print(c)


print("e1")
print(e1.shape)
print(e1)
print()

print("e2")
print(e2.shape)
print(e2)
print()

print("e3")
print(e3.shape)
print(e3)
print()


(2, 3)
(7,)
a
[[0 1 2]
 [3 4 5]]
c
[0 1 2 3 4 5 6]
e1
(2, 3, 7)
[[[ 0  0  0  0  0  0  0]
  [ 0  1  2  3  4  5  6]
  [ 0  2  4  6  8 10 12]]

 [[ 0  3  6  9 12 15 18]
  [ 0  4  8 12 16 20 24]
  [ 0  5 10 15 20 25 30]]]

e2
(2, 7, 3)
[[[ 0  0  0]
  [ 0  1  2]
  [ 0  2  4]
  [ 0  3  6]
  [ 0  4  8]
  [ 0  5 10]
  [ 0  6 12]]

 [[ 0  0  0]
  [ 3  4  5]
  [ 6  8 10]
  [ 9 12 15]
  [12 16 20]
  [15 20 25]
  [18 24 30]]]

e3
(7, 2, 3)
[[[ 0  0  0]
  [ 0  0  0]]

 [[ 0  1  2]
  [ 3  4  5]]

 [[ 0  2  4]
  [ 6  8 10]]

 [[ 0  3  6]
  [ 9 12 15]]

 [[ 0  4  8]
  [12 16 20]]

 [[ 0  5 10]
  [15 20 25]]

 [[ 0  6 12]
  [18 24 30]]]



In [65]:
#Functions
def l(array):
    return np.roll(array,1,axis=-1)

def ddx(array, ddom):
    return (array - l(array))/ddom

# Parameters, Domains, etc.
Xpts = 100
L = 10
X = np.linspace(0,L,Xpts+1)
dx = L/Xpts

num_dom_ext = 5
Xppts = num_dom_ext * Xpts
Xp = np.linspace(-2*L,3*L-dx, Xppts)

Wpts = 300
WL = 30
dw = WL/Wpts
Wp = np.linspace(0,WL,Wpts)+dw

mean_n = 3/(4*np.pi)
q = 1

# Memory Allocation - Only Correlations
n = np.zeros(Xpts)
temp = np.zeros(Xpts)
I = np.zeros((Xpts, Wpts, Xppts))
n_p = np.zeros(Xppts)
beta_p = np.zeros(Xppts)

# Pretend Instantiation Of Values
n = np.arange(Xpts)
temp = np.arange(Xpts) + 1

st_time = time.time()

# Periodic Function Extensions - Extend x to x'
# Shape: (Xppts,)
n_p = np.tile(n, num_dom_ext)
beta_p = np.tile(1 / temp, num_dom_ext)

# Extend x and x' to (x,x')
# Shape: (Xpts, Xppts)
n_x_p = np.subtract.outer(n,n_p) - mean_n
beta_x_p = np.stack([beta_p]*Xpts, axis = 0)

# Extend (x,x') to (x,x',w')
# Shape: (Xpts, Xppts, Wpts)
n_p_wp = np.stack([n_x_p]*Wpts, axis = 1)
beta_p_wp = np.stack([beta_x_p]*Wpts, axis = 1)

# Define I function
# Shape: (Xpts, Xppts, Wpts)
u_potential = beta_p_wp * (1/Wp)[:,None]
I = (1-np.exp(-q**2 * u_potential))*(ddx(beta_p_wp, dx)-u_potential*Xp)

# Calculate full integral
corr_integrand = n_p_wp*I
first_int = np.trapz(corr_integrand, axis=-1, dx=dx)  # Integrate over x'
res = np.trapz(first_int, axis=-1, dx=dw)  # Integrate over w'

end_time = time.time()

print(res)
print(end_time-st_time)


[ 61.35977409  65.58449661  69.80921913  74.03394166  78.25866418
  82.4833867   86.70810922  90.93283174  95.15755426  99.38227678
 103.6069993  107.83172182 112.05644435 116.28116687 120.50588939
 124.73061191 128.95533443 133.18005695 137.40477947 141.62950199
 145.85422452 150.07894704 154.30366956 158.52839208 162.7531146
 166.97783712 171.20255964 175.42728216 179.65200468 183.87672721
 188.10144973 192.32617225 196.55089477 200.77561729 205.00033981
 209.22506233 213.44978485 217.67450738 221.8992299  226.12395242
 230.34867494 234.57339746 238.79811998 243.0228425  247.24756502
 251.47228754 255.69701007 259.92173259 264.14645511 268.37117763
 272.59590015 276.82062267 281.04534519 285.27006771 289.49479024
 293.71951276 297.94423528 302.1689578  306.39368032 310.61840284
 314.84312536 319.06784788 323.2925704  327.51729293 331.74201545
 335.96673797 340.19146049 344.41618301 348.64090553 352.86562805
 357.09035057 361.31507309 365.53979562 369.76451814 373.98924066
 378.213963